# Visualize DDoS Attack

In [ ]:
import pandas as pd
import os
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
http_flood_file_relative_path = '../data/Edge-IIoTset dataset/Attack traffic/DDoS_HTTP_Flood_attack.csv'

http_flood_file = os.path.abspath(http_flood_file_relative_path)
# Read the CSV file
http_flood_df = pd.read_csv(http_flood_file)

# Display the data
http_flood_df.head(5)

In [ ]:
# Create a directed graph
G = nx.DiGraph()

filtered_df = http_flood_df[
    (http_flood_df['ip.src_host'] != '0.0.0.0') &
    (http_flood_df['ip.src_host'] != '0') &
    (http_flood_df['ip.dst_host'] != '224.0.0.1') &
    (http_flood_df['ip.dst_host'] != '0')
]

# Get the count of connections for each source node
node_sizes = http_flood_df['ip.src_host'].value_counts()

# Add edges from 'ip.src_host' to 'ip.dst_host'
for index, row in filtered_df.iterrows():
    src_host = row['ip.src_host']
    dst_host = row['ip.dst_host']
    dst_port = row['tcp.dstport']
    if src_host not in G.nodes():
        G.add_node(src_host, size=node_sizes[src_host])
    if G.has_edge(src_host, dst_host):
        G[src_host][dst_host]['count'] += 1
    else:
        G.add_edge(src_host, dst_host, count=1, dst_port=str('Port: ' + str(int(dst_port))))


# Plot the graph with one-directional edges and tcp.dstport labels
plt.figure(figsize=(30, 30))
pos = nx.spring_layout(G)
nx.draw_networkx(G, pos, node_size=[node_sizes.get(node, 1) * 1.1 for node in G.nodes()], with_labels=True, font_size=12, arrows=True)
edge_labels = nx.get_edge_attributes(G, 'dst_port')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.title('HTTP Flood Connections', fontsize=36)
plt.show()